In [1]:
# Eğer bu kütüphaneler kurulu değilse aşağıdaki komutları terminalde çalıştırabilirsiniz:
# pip install rasterio geopandas numpy pandas tqdm

import os
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from tqdm import tqdm

print("Gerekli kütüphaneler yüklendi.")


Gerekli kütüphaneler yüklendi.


In [5]:

# Veri yollarını tanımla
RAW_DIR = Path("../data/raw")
PROC_DIR = Path("../data/processed")
EXTERNAL_DIR = Path("../data/external")  # düzeltme: external raw içinde değil, onunla aynı seviyede

ELEV_DIR = RAW_DIR / "elevation"
LIGHT_PATH = RAW_DIR / "lightpollution" / "VNL_npp_2024_global_vcmslcfg_v2_c202502261200.median_masked.dat.tif"
BOUNDARY_PATH = EXTERNAL_DIR / "boundaries" / "turkey_boundary.geojson"  # düzeltildi

# İşlenmiş dosyaların kayıt edileceği yollar
ELEV_MOSAIC_PATH = PROC_DIR / "srtm_turkiye_mosaic.tif"
ELEV_CROPPED_PATH = PROC_DIR / "srtm_turkiye_cropped.tif"
LIGHT_CROPPED_PATH = PROC_DIR / "viirs_light_2024_turkey.tif"

# İşlenmiş veri klasörü varsa oluştur
PROC_DIR.mkdir(parents=True, exist_ok=True)

print("Dosya yolları başarıyla tanımlandı.")
print(f"TÜRKİYE SINIRI: {BOUNDARY_PATH.exists() = }, yol: {BOUNDARY_PATH}")
print(f"IŞIK VERİSİ: {LIGHT_PATH.exists() = }, yol: {LIGHT_PATH}")
print(f"YÜKSEKLİK VERİSİ DİZİNİ: {ELEV_DIR.exists() = }, yol: {ELEV_DIR}")


Dosya yolları başarıyla tanımlandı.
TÜRKİYE SINIRI: BOUNDARY_PATH.exists() = True, yol: ..\data\external\boundaries\turkey_boundary.geojson
IŞIK VERİSİ: LIGHT_PATH.exists() = True, yol: ..\data\raw\lightpollution\VNL_npp_2024_global_vcmslcfg_v2_c202502261200.median_masked.dat.tif
YÜKSEKLİK VERİSİ DİZİNİ: ELEV_DIR.exists() = True, yol: ..\data\raw\elevation


In [6]:
# Türkiye sınırlarını GeoDataFrame olarak yükle
gdf_turkey = gpd.read_file(BOUNDARY_PATH)
gdf_turkey = gdf_turkey.to_crs("EPSG:4326")  # Raster veriler genelde WGS84 kullanır

# GeoJSON geometrisini rasterio ile kullanmak için GeoJSON formatına çevir
from shapely.geometry import mapping
turkey_geom = [mapping(geom) for geom in gdf_turkey.geometry]

print("Türkiye sınırları yüklendi ve uygun projeksiyona dönüştürüldü.")


Türkiye sınırları yüklendi ve uygun projeksiyona dönüştürüldü.


In [7]:
# Tüm SRTM dosyalarını oku ve birleştir (mozaik)
elev_files = list(ELEV_DIR.glob("*.tif"))
src_files_to_mosaic = []

for fp in tqdm(elev_files, desc="SRTM dosyaları okunuyor"):
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# Mozaik işlemi
mosaic, out_transform = merge(src_files_to_mosaic)

# Mozaik dosyanın metadata bilgisi
out_meta = src.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform,
    "crs": "EPSG:4326"
})

# Mozaik rasteri diske yaz
with rasterio.open(ELEV_MOSAIC_PATH, "w", **out_meta) as dest:
    dest.write(mosaic)

print(f"Yükseklik verisi mozaiklenerek {ELEV_MOSAIC_PATH.name} dosyasına kaydedildi.")


SRTM dosyaları okunuyor: 100%|██████████| 172/172 [00:01<00:00, 102.47it/s]


Yükseklik verisi mozaiklenerek srtm_turkiye_mosaic.tif dosyasına kaydedildi.


In [8]:
# Mozaiklenmiş rasterı oku
with rasterio.open(ELEV_MOSAIC_PATH) as src:
    out_image, out_transform = mask(src, turkey_geom, crop=True)
    out_meta = src.meta.copy()

# Metadata güncelle
out_meta.update({
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Kırpılmış rasterı kaydet
with rasterio.open(ELEV_CROPPED_PATH, "w", **out_meta) as dest:
    dest.write(out_image)

print(f"Kırpılmış yükseklik verisi {ELEV_CROPPED_PATH.name} dosyasına kaydedildi.")


Kırpılmış yükseklik verisi srtm_turkiye_cropped.tif dosyasına kaydedildi.


In [9]:
# VIIRS rasterını oku ve Türkiye sınırına kırp
with rasterio.open(LIGHT_PATH) as src:
    light_crop, light_transform = mask(src, turkey_geom, crop=True)
    light_meta = src.meta.copy()

# Metadata güncelle
light_meta.update({
    "height": light_crop.shape[1],
    "width": light_crop.shape[2],
    "transform": light_transform
})

# Kırpılmış ışık rasterını kaydet
with rasterio.open(LIGHT_CROPPED_PATH, "w", **light_meta) as dest:
    dest.write(light_crop)

print(f"Kırpılmış ışık verisi {LIGHT_CROPPED_PATH.name} dosyasına kaydedildi.")


Kırpılmış ışık verisi viirs_light_2024_turkey.tif dosyasına kaydedildi.


In [10]:
# Sayısal analiz fonksiyonu
def analyze_raster(path, desc):
    with rasterio.open(path) as src:
        data = src.read(1)
        masked_data = np.ma.masked_equal(data, src.nodata)
        print(f"\n--- {desc} ---")
        print(f"Min: {masked_data.min()}")
        print(f"Max: {masked_data.max()}")
        print(f"Mean: {masked_data.mean():.2f}")
        print(f"Shape: {masked_data.shape}")
        print(f"Nodata: {src.nodata}")

# Yükseklik rasterı analizi
analyze_raster(ELEV_CROPPED_PATH, "Rakım (Yükseklik) Verisi")

# Işık kirliliği rasterı analizi
analyze_raster(LIGHT_CROPPED_PATH, "Işık Kirliliği Verisi")



--- Rakım (Yükseklik) Verisi ---
Min: -38
Max: 5112
Mean: 1139.78
Shape: (22650, 69013)
Nodata: -32767.0

--- Işık Kirliliği Verisi ---
Min: 0.0
Max: 320.30999755859375
Mean: 0.98
Shape: (1511, 4601)
Nodata: None


In [16]:
def compute_histogram(path, bins=50, desc="Raster"):
    with rasterio.open(path) as src:
        data = src.read(1)
        masked_data = np.ma.masked_equal(data, src.nodata)

        # Histogram hesapla
        hist, bin_edges = np.histogram(masked_data.compressed(), bins=bins)

        print(f"\n--- {desc} Histogram ---")
        for i in range(len(hist)):
            print(f"{bin_edges[i]:8.2f} - {bin_edges[i+1]:8.2f}: {hist[i]} piksel")

        return hist, bin_edges

# Yükseklik verisi için histogram
compute_histogram(ELEV_CROPPED_PATH, bins=10, desc="Rakım (Yükseklik)")

# Işık kirliliği verisi için histogram
compute_histogram(LIGHT_CROPPED_PATH, bins=10, desc="Işık Kirliliği")



--- Rakım (Yükseklik) Histogram ---
  -38.00 -   477.00: 179466363 piksel
  477.00 -   992.00: 262317640 piksel
  992.00 -  1507.00: 318524406 piksel
 1507.00 -  2022.00: 184985145 piksel
 2022.00 -  2537.00: 82083864 piksel
 2537.00 -  3052.00: 22702230 piksel
 3052.00 -  3567.00: 2733893 piksel
 3567.00 -  4082.00: 72412 piksel
 4082.00 -  4597.00: 14316 piksel
 4597.00 -  5112.00: 6417 piksel

--- Işık Kirliliği Histogram ---
    0.00 -    32.03: 6911415 piksel
   32.03 -    64.06: 29047 piksel
   64.06 -    96.09: 10209 piksel
   96.09 -   128.12: 1120 piksel
  128.12 -   160.15: 215 piksel
  160.15 -   192.19: 69 piksel
  192.19 -   224.22: 21 piksel
  224.22 -   256.25: 8 piksel
  256.25 -   288.28: 5 piksel
  288.28 -   320.31: 2 piksel


(array([6911415,   29047,   10209,    1120,     215,      69,      21,
              8,       5,       2], dtype=int64),
 array([  0.   ,  32.031,  64.062,  96.093, 128.124, 160.155, 192.186,
        224.217, 256.248, 288.279, 320.31 ], dtype=float32))

In [17]:
def describe_raster(path, desc="Raster"):
    with rasterio.open(path) as src:
        data = src.read(1)
        masked_data = np.ma.masked_equal(data, src.nodata)
        values = masked_data.compressed()

        stats = {
            "Min": values.min(),
            "Max": values.max(),
            "Mean": values.mean(),
            "Std": values.std(),
            "25%": np.percentile(values, 25),
            "50% (Median)": np.percentile(values, 50),
            "75%": np.percentile(values, 75),
            "90%": np.percentile(values, 90),
            "95%": np.percentile(values, 95),
            "99%": np.percentile(values, 99),
        }

        print(f"\n--- {desc} İstatistikleri ---")
        for k, v in stats.items():
            print(f"{k:<12}: {v:>8.2f}")
        return stats

describe_raster(ELEV_CROPPED_PATH, desc="Rakım (Yükseklik)")
describe_raster(LIGHT_CROPPED_PATH, desc="Işık Kirliliği")




--- Rakım (Yükseklik) İstatistikleri ---
Min         :   -38.00
Max         :  5112.00
Mean        :  1139.78
Std         :   654.87
25%         :   689.00
50% (Median):  1090.00
75%         :  1575.00
90%         :  2031.00
95%         :  2301.00
99%         :  2778.00

--- Işık Kirliliği İstatistikleri ---
Min         :     0.00
Max         :   320.31
Mean        :     0.98
Std         :     4.98
25%         :     0.00
50% (Median):     0.00
75%         :     0.65
90%         :     1.55
95%         :     3.01
99%         :    18.73


{'Min': 0.0,
 'Max': 320.31,
 'Mean': 0.9803241,
 'Std': 4.978313,
 '25%': 0.0,
 '50% (Median)': 0.0,
 '75%': 0.6462754905223846,
 '90%': 1.5545257329940796,
 '95%': 3.005101799964905,
 '99%': 18.72749061584478}

In [3]:
import pandas as pd
from pathlib import Path

# Dosya yolu
csv_path = Path("../data/processed/climate/merged_climate_data.csv")

# CSV dosyasını oku
df = pd.read_csv(csv_path)

# --- Temel Bilgiler ---
print("🧾 Veri Kümesi Genel Bilgi:\n")
print(df.info())

# --- İlk 5 Satırı Göster ---
print("\n🔍 İlk 5 Satır:\n")
print(df.head())

# --- Sütun İsimleri ---
print("\n🧭 Sütunlar:\n")
print(df.columns.tolist())

# --- Sayısal Sütunlardan Özet İstatistikler ---
print("\n📊 Temel İstatistikler:\n")
print(df.describe().T)

# --- Eksik Veri Kontrolü ---
print("\n⚠️ Eksik Veri Sayısı:\n")
print(df.isna().sum()[df.isna().sum() > 0])


🧾 Veri Kümesi Genel Bilgi:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4173 entries, 0 to 4172
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   latitude                 4173 non-null   float64
 1   longitude                4173 non-null   float64
 2   ALLSKY_SFC_PAR_TOT_grp1  4173 non-null   float64
 3   ALLSKY_SFC_SW_DWN_grp1   4173 non-null   float64
 4   CLOUD_AMT_grp1           4173 non-null   float64
 5   CLOUD_AMT_DAY_grp1       4173 non-null   float64
 6   CLOUD_AMT_NIGHT_grp1     4173 non-null   float64
 7   CLRSKY_DAYS_grp1         4173 non-null   float64
 8   CLRSKY_SFC_SW_DWN_grp1   4173 non-null   float64
 9   QV2M_grp2                4173 non-null   float64
 10  RH2M_grp2                4173 non-null   float64
 11  T2M_grp2                 4173 non-null   float64
 12  T2MDEW_grp2              4173 non-null   float64
 13  T2MWET_grp2              4173 non-null   float64
 

---

| Parametre Adı           | Anlamı                                | Birim      | Grup   | Count | Mean     | Std      | Min    | 25%    | 50%    | 75%    | Max    |
|------------------------|----------------------------------------|------------|--------|-------|----------|----------|--------|--------|--------|--------|--------|
| latitude               | Enlem                                  | °          | -      | 4173  | 39.0685  | 1.3877   | 36.00  | 38.00  | 39.00  | 40.00  | 42.00  |
| longitude              | Boylam                                 | °          | -      | 4173  | 35.2259  | 4.8791   | 26.50  | 31.00  | 35.00  | 39.00  | 45.00  |
| ALLSKY_SFC_PAR_TOT_grp1| Fotosentetik aktif radyasyon (toplam)  | MJ/m²/gün  | grp1   | 4173  | 2.0301   | 0.8603   | 0.56   | 1.23   | 1.86   | 2.91   | 4.56   |
| ALLSKY_SFC_SW_DWN_grp1 | Tüm gökyüzü kısa dalga radyasyon       | kWh/m²/gün | grp1   | 4173  | 4.5936   | 1.9343   | 1.26   | 2.81   | 4.33   | 6.50   | 8.36   |
| CLOUD_AMT_grp1         | Ortalama bulutluluk oranı              | %          | grp1   | 4173  | 53.3564  | 18.5931  | 6.38   | 40.99  | 55.79  | 67.43  | 92.38  |
| CLOUD_AMT_DAY_grp1     | Gündüz bulutluluk oranı                | %          | grp1   | 4173  | 53.5030  | 17.7185  | 8.16   | 42.17  | 56.67  | 66.66  | 89.41  |
| CLOUD_AMT_NIGHT_grp1   | Gece bulutluluk oranı                  | %          | grp1   | 4173  | 52.7377  | 19.8702  | 3.50   | 40.59  | 56.20  | 68.97  | 93.12  |
| CLRSKY_DAYS_grp1       | Açık gün sayısı (yılda)                | gün        | grp1   | 4173  | 121.9842 | 258.9803 | -999.0 | 13.00  | 80.00  | 171.00 | 1465.0 |
| CLRSKY_SFC_SW_DWN_grp1 | Açık gökyüzü kısa dalga radyasyon      | kWh/m²/gün | grp1   | 4173  | 5.6916   | 1.9456   | 2.08   | 4.03   | 5.57   | 7.24   | 9.93   |
| QV2M_grp2              | 2m özgül nem                           | g/kg       | grp2   | 4173  | 6.0373   | 2.1705   | 2.01   | 4.36   | 5.54   | 7.37   | 12.53  |
| RH2M_grp2              | 2m bağıl nem                           | %          | grp2   | 4173  | 65.6405  | 14.4074  | 18.70  | 57.46  | 66.15  | 74.65  | 87.48  |
| T2M_grp2               | 2m sıcaklık                            | °C         | grp2   | 4173  | 11.3188  | 8.9191   | -10.21 | 4.54   | 10.69  | 17.53  | 30.38  |
| T2MDEW_grp2            | Çiy noktası sıcaklığı                  | °C         | grp2   | 4173  | 3.4315   | 5.6934   | -12.23 | -0.57  | 2.06   | 6.70   | 18.86  |
| T2MWET_grp2            | Islak termometre sıcaklığı             | °C         | grp2   | 4173  | 7.3751   | 7.0924   | -10.94 | 2.07   | 6.28   | 11.81  | 24.00  |
| T2M_MAX_grp2           | Maksimum sıcaklık                      | °C         | grp2   | 4173  | 28.9018  | 9.2579   | 3.17   | 21.71  | 28.32  | 36.26  | 51.21  |
| T2M_MIN_grp2           | Minimum sıcaklık                       | °C         | grp2   | 4173  | -6.2601  | 11.8247  | -33.30 | -16.26 | -5.77  | 2.67   | 24.09  |
| T2M_RANGE_grp2         | Günlük sıcaklık farkı                  | °C         | grp2   | 4173  | 11.6682  | 2.7755   | 3.23   | 9.63   | 11.50  | 13.61  | 20.37  |
| TQV_grp2               | Toplam su buharı                       | kg/m²      | grp2   | 4173  | 13.0420  | 4.9739   | 3.46   | 9.21   | 12.27  | 16.02  | 29.94  |
| PS_grp3                | Yüzey basıncı                          | kPa        | grp3   | 4173  | 88.9133  | 6.0195   | 74.69  | 84.68  | 88.79  | 93.62  | 106.36 |
| SLP_grp3               | Deniz seviyesi basıncı                 | kPa        | grp3   | 4173  | 101.3652 | 0.4274   | 99.94  | 101.12 | 101.37 | 101.67 | 102.38 |
| WD2M_grp3              | Rüzgar yönü (2m)                       | °          | grp3   | 4173  | 176.9109 | 112.2666 | 0.10   | 60.10  | 190.00 | 290.00 | 359.90 |
| WS2M_grp3              | Rüzgar hızı (2m)                       | m/s        | grp3   | 4173  | 2.2083   | 0.5585   | 0.33   | 1.90   | 2.13   | 2.47   | 5.32   |
| WS2M_MAX_grp3          | Maksimum rüzgar hızı (2m)              | m/s        | grp3   | 4173  | 9.5040   | 2.5556   | 1.23   | 7.73   | 9.43   | 10.97  | 20.02  |
| PRECTOTCORR_grp4       | Toplam düzeltilmiş yağış               | mm/gün     | grp4   | 4173  | 1.5164   | 0.9214   | 0.00   | 0.96   | 1.36   | 1.95   | 5.19   |
| TO3_grp4               | Toplam ozon                            | Dobson     | grp4   | 4173  | 312.0737 | 24.6936  | 271.36 | 287.48 | 308.12 | 334.91 | 367.66 |
| TS_grp4                | Yüzey sıcaklığı                        | °C         | grp4   | 4173  | 11.5270  | 9.8124    | -13.98 | 4.20   |10.12  | 17.52  | 32.14  |
 12.53   |
